<a href="https://colab.research.google.com/github/QuantumZeroAR/Capacitacion_AI/blob/main/C8_0_Red_Neuronal_Convolucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
datos, metadatos = tfds.load("mnist", as_supervised=True, with_info=True)

In [ ]:
metadatos

In [4]:
datos_entrenamiento = datos["train"]
datos_pruebas = datos["test"]

In [5]:
nombres_clases = metadatos.features["label"].names

In [6]:
#Normalizacion
#0-255 1 byte
#Transformar 0-255 0-1 (0=0, 255=1, 127=.5)

#Normalizar datos ent y pruebas con una funcion
def normalizar(imagenes, etiquetas):
  imagenes = tf.cast(imagenes, tf.float32)
  imagenes = imagenes/255
  return imagenes, etiquetas

datos_entrenamiento = datos_entrenamiento.map(normalizar)
datos_pruebas = datos_pruebas.map(normalizar)

#Agregar en cache
datos_entrenamiento = datos_entrenamiento.cache()
datos_pruebas = datos_pruebas.cache()

In [ ]:
for imagen, etiqueta in datos_entrenamiento.take(1):
  break

import matplotlib.pyplot as plt

#Mostrarlo en pantalla
plt.figure()
plt.imshow(imagen, cmap=plt.cm.binary)
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
for i, (imagen, etiqueta) in enumerate(datos_entrenamiento.take(25)):
  #Mostrarlo en pantalla
  plt.subplot(5, 5, i+1)
  plt.imshow(imagen, cmap=plt.cm.binary)

plt.show()

In [ ]:
"""modelo = tf.keras.Sequential([
    #784 datos de entrada
    tf.keras.layers.Flatten(input_shape=(28,28,1)), #1 = blanco y negro
    # # # Capa oculta
    tf.keras.layers.Dense(50, activation=tf.nn.relu),
    tf.keras.layers.Dense(50, activation=tf.nn.relu),
    # Capa de salida
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])"""

modelo = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), input_shape=(28,28,1), activation="relu"), #Primera capa de convolucion. Agrego 32 filtros de 3x3
    tf.keras.layers.MaxPooling2D(2,2),                                           #Agrego capa de agrupacion

    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),                       #Segunda capa de convolucion. Agrego 64 filtros de 3x3
    tf.keras.layers.MaxPooling2D(2,2),                                          #Agrego capa de agrupacion

    tf.keras.layers.Flatten(),                                                  #Aca lo estoy agregando lo anterior en una red densa regular
    tf.keras.layers.Dense(units=100, activation="relu"),                        #Capa oculta de 100 neuronas con activacion Relu
    tf.keras.layers.Dense(10, activation="softmax")                             #10 neuronas de saludas con actuvacion softmax para catalogar.
])

In [10]:
modelo.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [11]:
TAMANO_LOTE = 32

datos_entrenamiento = datos_entrenamiento.repeat().shuffle(60000).batch(TAMANO_LOTE)
datos_pruebas = datos_pruebas.batch(TAMANO_LOTE)

In [ ]:
import math

#Entrenar
entrnamiento = modelo.fit(
    datos_entrenamiento, epochs=10,
    steps_per_epoch=math.ceil(60000/TAMANO_LOTE)
)

In [ ]:
#Pintar una cuadricula con varias predicciones, y marcar si fue correcta (azul) o incorrecta (roja)
import numpy as np

for imagenes_prueba, etiquetas_prueba in datos_pruebas.take(1):
  imagenes_prueba = imagenes_prueba.numpy()
  etiquetas_prueba = etiquetas_prueba.numpy()
  predicciones = modelo.predict(imagenes_prueba)

def graficar_imagen(i, arr_predicciones, etiquetas_reales, imagenes):
  arr_predicciones, etiqueta_real, img = arr_predicciones[i], etiquetas_reales[i], imagenes[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img[...,0], cmap=plt.cm.binary)

  etiqueta_prediccion = np.argmax(arr_predicciones)
  if etiqueta_prediccion == etiqueta_real:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(nombres_clases[etiqueta_prediccion],
                                100*np.max(arr_predicciones),
                                nombres_clases[etiqueta_real]),
                                color=color)

def graficar_valor_arreglo(i, arr_predicciones, etiqueta_real):
  arr_predicciones, etiqueta_real = arr_predicciones[i], etiqueta_real[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  grafica = plt.bar(range(10), arr_predicciones, color="#777777")
  plt.ylim([0, 1])
  etiqueta_prediccion = np.argmax(arr_predicciones)

  grafica[etiqueta_prediccion].set_color('red')
  grafica[etiqueta_real].set_color('blue')

filas = 5
columnas = 5
num_imagenes = filas*columnas
plt.figure(figsize=(2*2*columnas, 2*filas))
for i in range(num_imagenes):
  plt.subplot(filas, 2*columnas, 2*i+1)
  graficar_imagen(i, predicciones, etiquetas_prueba, imagenes_prueba)
  plt.subplot(filas, 2*columnas, 2*i+2)
  graficar_valor_arreglo(i, predicciones, etiquetas_prueba)

In [ ]:
datos, metadatos = tfds.load("cats_vs_dogs", as_supervised=True, with_info=True)

In [ ]:
metadatos

In [ ]:
tfds.show_examples(datos["train"], metadatos)

In [ ]:
#Vamos a rimensionar las imagenes a un mismo volor de medido 100x100 y las paso a Escalas de grises para que sea mas facil procesar
import matplotlib.pyplot as plt
import cv2 #opencv - trabajar y hacer trnasformaciones en imagenes

plt.figure(figsize=(20,20))

TAMANO_IMG=100

for i, (imagen, etiqueta) in enumerate(datos["train"].take(25)):
  imagen = cv2.resize(imagen.numpy(), (TAMANO_IMG, TAMANO_IMG))
  imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
  plt.subplot(5, 5, i+1)
  plt.imshow(imagen, cmap="gray")

In [18]:
datos_entrenamiento = []

In [19]:
#Vamos a iterar para cargar todas las imagenes con el nuevo formato
for i, (imagen, etiqueta) in enumerate(datos["train"]):
  imagen = cv2.resize(imagen.numpy(), (TAMANO_IMG, TAMANO_IMG))
  imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
  imagen = imagen.reshape(TAMANO_IMG, TAMANO_IMG, 1)
  datos_entrenamiento.append([imagen, etiqueta])

In [ ]:
datos_entrenamiento[0]

In [21]:
#Preparar variablas X y y imagnes y etiquetas

X = []
y = []

for imagen, etiqueta in datos_entrenamiento:
  X.append(imagen)
  y.append(etiqueta)

In [22]:
#Normalizar las imagenes
import numpy as np

X = np.array(X).astype(float) / 255

In [ ]:
X[0]

In [26]:
y

array([1, 1, 1, ..., 0, 1, 0])

In [25]:
y = np.array(y)

In [27]:
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(100, 100, 1)),  #Primer capa de convolucion. Agrego 32 filtros de 3x3
    tf.keras.layers.MaxPooling2D(2, 2),                                               #Agrego capa de agrupacion de 2x2
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),                             #Segunda capa de convolucion. Agrego 64 filtros de 3x3
    tf.keras.layers.MaxPooling2D(2, 2),                                               #Agrego capa de agrupacion de 2x2
    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),                            #Tercer capa de convolucion. Agrego 128 filtros de 3x3
    tf.keras.layers.MaxPooling2D(2, 2),                                               #Agrego capa de agrupacion de 2x2
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation="relu"),                                    #Capa densa de 100 neuronas como primer ejemplo
    tf.keras.layers.Dense(1, activation="sigmoid")                                    #La salida solo con 1 neurona y funcion de activacion sigmoid
                                                                                      #fuerza una salida de 0 a 1 (o es perro o es gato)
])

In [28]:
modelo.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)

In [ ]:
modelo.fit(
    X, y, batch_size=32, validation_split=0.15, epochs=50
)

In [ ]:
#Sobreajuste over-fitting

#Dropout, apagar neuronas
#Aumento de datos